In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords

## Intro to word2vec

The most common unsupervised neural network approach for NLP is word2vec, a shallow neural network model for converting words to vectors using distributed representation: Each word is represented by many neurons, and each neuron is involved in representing many words.  At the highest level of abstraction, word2vec assigns a vector of random values to each word.  For a word W, it looks at the words that are near W in the sentence, and shifts the values in the word vectors such that the vectors for words near that W are closer to the W vector, and vectors for words not near W are farther away from the W vector.  With a large enough corpus, this will eventually result in words that often appear together having vectors that are near one another, and words that rarely or never appear together having vectors that are far away from each other.  Then, using the vectors, similarity scores can be computed for each pair of words by taking the cosine of the vectors.  

This may sound quite similar to the Latent Semantic Analysis approach you just learned.  The conceptual difference is that LSA creates vector representations of sentences based on the words in them, while word2vec creates representations of individual words, based on the words around them.

## What is it good for?

Word2vec is useful for any time when computers need to parse requests written by humans. The problem with human communication is that there are so many different ways to communicate the same concept. It's easy for us, as humans, to know that "the silverware" and "the utensils" can refer to the same thing. Computers can't do that unless we teach them, and this can be a real chokepoint for human/computer interactions. If you've ever played a text adventure game (think _Colossal Cave Adventure_ or _Zork_), you may have encountered the following scenario:

And your brain explodes from frustration. A text adventure game that incorporates a properly trained word2vec model would have vectors for "pick up", "lift", and "take" that are close to the vector for "grab" and therefore could accept those other verbs as synonyms so you could move ahead faster. In more practical applications, word2vec and other similar algorithms are what help a search engine return the best results for your query and not just the ones that contain the exact words you used. In fact, search is a better example, because not only does the search engine need to understand your request, it also needs to match it to web pages that were _also written by humans_ and therefore _also use idiosyncratic language_.

Humans, man.  

So how does it work?

## Generating vectors: Multiple algorithms

In considering the relationship between a word and its surrounding words, word2vec has two options that are the inverse of one another:

 * _Continuous Bag of Words_ (CBOW): the identity of a word is predicted using the words near it in a sentence.
 * _Skip-gram_: The identities of words are predicted from the word they surround. Skip-gram seems to work better for larger corpuses.

For the sentence "Terry Gilliam is a better comedian than a director", if we focus on the word "comedian" then CBOW will try to predict "comedian" using "is", "a", "better", "than", "a", and "director".  Skip-gram will try to predict "is", "a", "better", "than", "a", and "director" using the word "comedian". In practice, for CBOW the vector for "comedian" will be pulled closer to the other words, while for skip-gram the vectors for the other words will be pulled closer to "comedian".  

In addition to moving the vectors for nearby words closer together, each time a word is processed some vectors are moved farther away. Word2vec has two approaches to "pushing" vectors apart:
 
 * _Negative sampling_: Like it says on the tin, each time a word is pulled toward some neighbors, the vectors for a randomly chosen small set of other words are pushed away.
 * _Hierarchical softmax_: Every neighboring word is pulled closer or farther from a subset of words chosen based on a tree of probabilities.

## What is similarity? Word2vec strengths and weaknesses

Keep in mind that word2vec operates on the assumption that frequent proximity indicates similarity, but words can be "similar" in various ways. They may be conceptually similar ("royal", "king", and "throne"), but they may also be functionally similar ("tremendous" and "negligible" are both common modifiers of "size"). Here is a more detailed exploration, [with examples](https://quomodocumque.wordpress.com/2016/01/15/messing-around-with-word2vec/), of what "similarity" means in word2vec.

One cool thing about word2vec is that it can identify similarities between words _that never occur near one another in the corpus_. For example, consider these sentences:

"The dog played with an elastic ball."
"Babies prefer the ball that is bouncy."
"I wanted to find a ball that's elastic."
"Tracy threw a bouncy ball."

"Elastic" and "bouncy" are similar in meaning in the text but don't appear in the same sentence. However, both appear near "ball". In the process of nudging the vectors around so that "elastic" and "bouncy" are both near the vector for "ball", the words also become nearer to one another and their similarity can be detected.

For a while after it was introduced, [no one was really sure why word2vec worked as well as it did](https://arxiv.org/pdf/1402.3722v1.pdf) (see last paragraph of the linked paper). A few years later, some additional math was developed to explain word2vec and similar models. If you are comfortable with both math and "academese", have a lot of time on your hands, and want to take a deep dive into the inner workings of word2vec, [check out this paper](https://arxiv.org/pdf/1502.03520v7.pdf) from 2016.  

One of the draws of word2vec when it first came out was that the vectors could be used to convert analogies ("king" is to "queen" as "man" is to "woman", for example) into mathematical expressions ("king" + "woman" - "man" = ?) and solve for the missing element ("queen"). This is kinda nifty.

A drawback of word2vec is that it works best with a corpus that is at least several billion words long. Even though the word2vec algorithm is speedy, this is a a lot of data and takes a long time! Our example dataset is only two million words long, which allows us to run it in the notebook without overwhelming the kernel, but probably won't give great results.  Still, let's try it!

There are a few word2vec implementations in Python, but the general consensus is the easiest one to us is in [gensim](https://radimrehurek.com/gensim/models/word2vec.html). Now is a good time to `pip install gensim` if you don't have it yet.

In [7]:
nltk.download('gutenberg')
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 74.9MB/s ta 0:00:01
  Running setup.py install for en-core-web-sm ... done

    Linking successful
    /opt/conda/lib/python3.7/site-packages/en_core_web_sm -->
    /opt/conda/lib/python3.7/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [5]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text[0:900000]


# Import all the Austen in the Project Gutenberg corpus.
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-' + novel + '.txt')
    austen = austen + work

# Clean the data.
austen_clean = text_cleaner(austen)

[nltk_data] Downloading package gutenberg to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [8]:
# Parse the data. This can take some time.
nlp = spacy.load('en')
austen_doc = nlp(austen_clean)

In [9]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences = []
for sentence in austen_doc.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)


print(sentences[20])
print('We have {} sentences and {} tokens.'.format(len(sentences), len(austen_clean)))

['for', 'daughter', 'eld', 'give', 'thing', 'tempt']
We have 7771 sentences and 900000 tokens.


In [10]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [11]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('however', 0.8316839933395386), ('henrietta', 0.7411789894104004), ('conscious', 0.7383444905281067), ('miss', 0.7320570945739746), ('room', 0.7277919054031372), ('support', 0.7228241562843323), ('god', 0.7173083424568176), ('consult', 0.6863638162612915), ('appearance', 0.685311496257782), ('oppose', 0.6777299046516418)]
0.7736473
dinner


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Clearly this model is not great – while some words given above might possibly fill in the analogy woman:lady::man:?, most answers likely make little sense. You'll notice as well that re-running the model likely gives you different results, indicating random chance plays a large role here.

We do, however, get a nice result on "marriage" being dissimilar to "breakfast", "lunch", and "dinner". 

## Drill 0

Take a few minutes to modify the hyperparameters of this model and see how its answers change. Can you wrangle any improvements?

In [12]:
# Tinker with hyperparameters here.
param_dict1 = {'workers':4, 'min_count':20, 'window':6, 'sg':0, 'sample':1e-3, 'size':300, 'hs':1} 
param_dict2 = {'workers':4, 'min_count':10, 'window':10, 'sg':0, 'sample':1e-3, 'size':300, 'hs':1}
param_dict3 = {'workers':4, 'min_count':10, 'window':6, 'sg':0, 'sample':1e-4, 'size':300, 'hs':1} 
param_dict4 = {'workers':4, 'min_count':10, 'window':6, 'sg':0, 'sample':1e-3, 'size':300, 'hs':0} 

In [13]:
model1 = word2vec.Word2Vec(sentences, **param_dict1)

# List of words in model.
vocab1 = model1.wv.vocab.keys()

print(model1.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model1.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model1.doesnt_match("breakfast marriage dinner lunch".split()))

[('goddard', 0.9166241884231567), ('cole', 0.888947069644928), ('dixon', 0.8764947652816772), ('miss', 0.8440383672714233), ('weston', 0.8357830047607422), ('benwick', 0.816769003868103), ('bates', 0.767813503742218), ('deal', 0.7410463094711304), ('wentworth', 0.7378031015396118), ('colonel', 0.7294687628746033)]
0.7446081
dinner


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  del sys.path[0]


In [14]:
model2 = word2vec.Word2Vec(sentences, **param_dict2)

# List of words in model.
vocab2 = model2.wv.vocab.keys()

print(model2.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model2.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model2.doesnt_match("breakfast marriage dinner lunch".split()))

[('benwick', 0.8432368040084839), ('mr', 0.8255596160888672), ('birth', 0.8172060251235962), ('advise', 0.8127955198287964), ('oppose', 0.7954429388046265), ('sofa', 0.7916456460952759), ('mary', 0.786056637763977), ('anne', 0.7814750671386719), ('manner', 0.781296968460083), ('excessively', 0.7811325788497925)]
0.69110703
marriage


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  del sys.path[0]


In [15]:
model3 = word2vec.Word2Vec(sentences, **param_dict3)

# List of words in model.
vocab3 = model3.wv.vocab.keys()

print(model3.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model3.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model3.doesnt_match("breakfast marriage dinner lunch".split()))

[('-pron-', 0.9998904466629028), ("'s", 0.9998884201049805), ('anne', 0.9998849034309387), ('little', 0.9998831748962402), ('till', 0.9998796582221985), ('sister', 0.9998788833618164), ('place', 0.9998781681060791), ('mr.', 0.9998776912689209), ('young', 0.9998763799667358), ('reach', 0.9998753070831299)]
0.99989784
dinner


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  del sys.path[0]


In [16]:
model4 = word2vec.Word2Vec(sentences, **param_dict4)

# List of words in model.
vocab4 = model4.wv.vocab.keys()

print(model4.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model4.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model4.doesnt_match("breakfast marriage dinner lunch".split()))

[('feel', 0.9992027282714844), ('wish', 0.9991961717605591), ('perfectly', 0.9991940259933472), ('sister', 0.9991859197616577), ('father', 0.9991848468780518), ('begin', 0.999182403087616), ('fear', 0.999181866645813), ('live', 0.9991793036460876), ('meet', 0.9991740584373474), ('pleased', 0.999173641204834)]
0.9991916
breakfast


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  del sys.path[0]


In [17]:
param_dict5 = {'workers':4, 'min_count':10, 'window':12, 'sg':0, 'sample':0.01, 'size':300, 'hs':1}

model5 = word2vec.Word2Vec(sentences, **param_dict5)

# List of words in model.
vocab5 = model5.wv.vocab.keys()

print(model5.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model5.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model5.doesnt_match("breakfast marriage dinner lunch".split()))

[('henrietta', 0.5845932960510254), ('mr', 0.5834046006202698), ('hayter', 0.5097928047180176), ('birth', 0.5004416108131409), ('anne', 0.49647271633148193), ('louisa', 0.49378907680511475), ('amusement', 0.4696272313594818), ('charles', 0.4688611626625061), ('appearance', 0.4546986222267151), ('sister', 0.45450836420059204)]
0.58054644
dinner


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  from ipykernel import kernelapp as app


Model 2 performed best, losing some similarity between Mr. and Mrs., but gaining the ability to say the woman:lady::man:mr and finding that marriage is the odd one out among the meals of the day.

# Example word2vec applications

You can use the vectors from word2vec as features in other models, or try to gain insight from the vector compositions themselves.

Here are some neat things people have done with word2vec:

 * [Visualizing word embeddings in Jane Austen's Pride and Prejudice](http://blogger.ghostweather.com/2014/11/visualizing-word-embeddings-in-pride.html). Skip to the bottom to see a _truly honest_ account of this data scientist's process.

 * [Tracking changes in Dutch Newspapers' associations with words like 'propaganda' and 'alien' from 1950 to 1990](https://www.slideshare.net/MelvinWevers/concepts-through-time-tracing-concepts-in-dutch-newspaper-discourse-using-sequential-word-vector-spaces).

 * [Helping customers find clothing items similar to a given item but differing on one or more characteristics](http://multithreaded.stitchfix.com/blog/2015/03/11/word-is-worth-a-thousand-vectors/).

## Drill 1: Word2Vec on 100B+ words

As we mentioned, word2vec really works best on a big corpus, but it can take half a day to clean such a corpus and run word2vec on it.  Fortunately, there are word2vec models available that have already been trained on _really_ big corpora. They are big files, but you can download a [pretrained model of your choice here](https://github.com/3Top/word2vec-api). At minimum, the ones built with word2vec (check the "Architecture" column) should load smoothly using an appropriately modified version of the code below, and you can play to your heart's content.

Because the models are so large, however, you may run into memory problems or crash the kernel. If you can't get a pretrained model to run locally, check out this [interactive web app of the Google News model](https://rare-technologies.com/word2vec-tutorial/#bonus_app) instead.

However you access it, play around with a pretrained model. Is there anything interesting you're able to pull out about analogies, similar words, or words that don't match? Write up a quick note about your tinkering and discuss it with your mentor during your next session.

In [18]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format ('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [19]:
# Play around with your pretrained model here.

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('fella', 0.6031545400619507), ('gentleman', 0.5849651098251343), ('chap', 0.5543248653411865), ('gent', 0.543907880783081), ('guy', 0.5265034437179565), ('lad', 0.5139425992965698), ('feller', 0.5072450041770935), ('bloke', 0.49030163884162903), ('rascal', 0.48736995458602905), ('ladies', 0.47617608308792114)]


In [20]:
print(model.wv.similarity('mr', 'mrs'))


0.66098833


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [21]:
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

marriage


/opt/conda/lib/python3.7/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [22]:
print(model.wv.most_similar(positive=['paper', 'brush'], negative=['pen']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('leaves_grass_clippings', 0.41287797689437866), ('bark_mulch', 0.40660426020622253), ('tree_prunings', 0.40122997760772705), ('grass_clippings_leaves', 0.39961177110671997), ('pine_needles', 0.3952524662017822), ('lawn_clippings', 0.39195531606674194), ('Hashimiyat_west', 0.39191994071006775), ('vegetative_debris', 0.3823920786380768), ('woody_debris', 0.38232386112213135), ('dryer_lint', 0.37130028009414673)]


In [23]:
print(model.wv.most_similar(positive=['paper', 'paintbrush'], negative=['pen']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('quilling', 0.43930816650390625), ('acrylic_paint', 0.43626928329467773), ('chalk_pastels', 0.42957842350006104), ('paintbrushes', 0.4273119270801544), ('pencil_watercolor', 0.4249844253063202), ('acrylic_paints', 0.42373189330101013), ('pencils_pens_markers', 0.41913706064224243), ('egg_tempera', 0.41786646842956543), ('sumi_ink', 0.4167347550392151), ('charcoal_pencils', 0.41660210490226746)]


In [24]:
print(model.wv.most_similar(positive=['paper', 'oil'], negative=['watercolor']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('petroleum', 0.5332932472229004), ('crude_oil', 0.5033786296844482), ('WSJ_sub_reqd', 0.48628664016723633), ('gas', 0.47056639194488525), ('crude', 0.4611533284187317), ('petro', 0.4544041156768799), ('natural_gas', 0.45328986644744873), ('hydrocarbon', 0.44675105810165405), ('Robert_Mabro', 0.41905832290649414), ('fossil_fuel', 0.4170653522014618)]


In [27]:
print(model.wv.most_similar(positive=['canvas', 'marble'], negative=['oil']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('mosaic_tiles', 0.5359538793563843), ('bas_relief_sculptures', 0.5348411798477173), ('canvases', 0.5303469896316528), ('marble_tiles', 0.5295888185501099), ('polychromed', 0.5283920764923096), ('marble_mosaic', 0.5193307399749756), ('bas_relief', 0.5139007568359375), ('Cor_Ten', 0.5122668743133545), ('Carrara_marble', 0.5104560852050781), ('newel_posts', 0.51004558801651)]


In [28]:
print(model.wv.most_similar(positive=['bun', 'fajita'], negative=['hamburger']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('sautéed_onions', 0.5586769580841064), ('chipotle_sauce', 0.5336236357688904), ('tomatillo_sauce', 0.533494770526886), ('roasted_poblano_peppers', 0.5304685235023499), ('roasted_poblano', 0.5229477882385254), ('mushrooms_onions', 0.5183507204055786), ('diced_tomato', 0.5180438160896301), ('flour_tortilla', 0.5150987505912781), ('sliced_jalapeños', 0.5131515264511108), ('lettuce_tomato_onion', 0.5098903179168701)]


In [30]:
print(model.wv.most_similar(positive=['bun', 'fajita'], negative=['hotdog']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('flour_tortilla', 0.5801827311515808), ('sautéed_onions', 0.5515615940093994), ('tomatillo_sauce', 0.5450605750083923), ('frittata', 0.5386663675308228), ('chipotle_sauce', 0.5343946218490601), ('fajitas', 0.5337242484092712), ('pepperjack_cheese', 0.5331733822822571), ('diced_tomato', 0.5328484773635864), ('roasted_poblano_peppers', 0.532839834690094), ('refried_beans', 0.5322811603546143)]


In [31]:
print(model.wv.most_similar(positive=['evening', 'brunch'], negative=['dinner']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('afternoon', 0.7141777276992798), ('night', 0.661408543586731), ('morning', 0.6369569301605225), ('evenings', 0.5446067452430725), ('Afternoon', 0.5420071482658386), ('Saturday', 0.5406385660171509), ('Sunday', 0.5403701066970825), ('afternoons', 0.524677038192749), ('noon', 0.5193315744400024), ('3pm_6pm', 0.5123345851898193)]


In [32]:
print(model.wv.most_similar(positive=['steeple', 'mosque'], negative=['church']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('minaret', 0.5984616875648499), ('tower', 0.5809642672538757), ('towers', 0.535079836845398), ('spire', 0.5302766561508179), ('twin_towers', 0.5160786509513855), ('minarets', 0.5052819848060608), ('brick_smokestack', 0.4848308861255646), ('turquoise_dome', 0.48442041873931885), ('tall_spiral_minaret', 0.4790385067462921), ('church_steeple', 0.47515785694122314)]


In [37]:
print(model.wv.most_similar(positive=['diamond', 'beryllium'], negative=['carbon']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('diamonds', 0.5379749536514282), ('gemstone', 0.48719996213912964), ('gemstones', 0.4563748240470886), ('NamGem', 0.44417282938957214), ('Diamonds', 0.4408671259880066), ('bearing_kimberlite', 0.43318212032318115), ('tantalite', 0.4285939633846283), ('Controller_Shmuel_Mordechai', 0.4253310561180115), ('miner_De_Beers', 0.42142707109451294), ('quartz', 0.4191233813762665)]


In [38]:
print(model.wv.most_similar(positive=['diamond', 'aluminum'], negative=['carbon']))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('diamonds', 0.5244714021682739), ('stainless_steel', 0.49729013442993164), ('titanium', 0.48880669474601746), ('aluminum_alloy', 0.47384119033813477), ('Smolensk_Kristall', 0.46902376413345337), ('alloy', 0.4656844437122345), ('sapphire', 0.4575681686401367), ('metal', 0.45581668615341187), ('gemstone', 0.4553333520889282), ('Aluminum', 0.45271608233451843)]
